In [18]:
import json
import os
import pandas as pd
import numpy as np
import asyncio
from eth_account import account
from web3 import Web3
from src import s3_services
from src import sql_manager
from datetime import datetime
from importlib import reload

In [19]:
#Web3 Connection Handler
projectID = '69765a3368f44f5c8cc3691d2361e496'

infura_url = f'https://mainnet.infura.io/v3/{projectID}'
web3 = Web3(Web3.HTTPProvider(infura_url))

if web3.isConnected():
    print('The connection with Ethereum network is successfull')
else:
    print('Network connection error')

The connection with Ethereum network is successfull


In [20]:
# Ronin Bridge Smart Contract
address = '0x1A2a1c938CE3eC39b6D47113c7955bAa9DD454F2'
abi = json.loads('[{"inputs":[{"internalType":"address","name":"_proxyTo","type":"address"},{"internalType":"address","name":"_registry","type":"address"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_oldAdmin","type":"address"},{"indexed":true,"internalType":"address","name":"_newAdmin","type":"address"}],"name":"AdminChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_oldAdmin","type":"address"}],"name":"AdminRemoved","type":"event"},{"anonymous":false,"inputs":[],"name":"Paused","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_new","type":"address"},{"indexed":true,"internalType":"address","name":"_old","type":"address"}],"name":"ProxyUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"_depositId","type":"uint256"},{"indexed":true,"internalType":"address","name":"_owner","type":"address"},{"indexed":true,"internalType":"address","name":"_tokenAddress","type":"address"},{"indexed":false,"internalType":"address","name":"_sidechainAddress","type":"address"},{"indexed":false,"internalType":"uint32","name":"_standard","type":"uint32"},{"indexed":false,"internalType":"uint256","name":"_tokenNumber","type":"uint256"}],"name":"TokenDeposited","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"_withdrawId","type":"uint256"},{"indexed":true,"internalType":"address","name":"_owner","type":"address"},{"indexed":true,"internalType":"address","name":"_tokenAddress","type":"address"},{"indexed":false,"internalType":"uint256","name":"_tokenNumber","type":"uint256"}],"name":"TokenWithdrew","type":"event"},{"anonymous":false,"inputs":[],"name":"Unpaused","type":"event"},{"payable":true,"stateMutability":"payable","type":"fallback"},{"constant":true,"inputs":[],"name":"admin","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"_newAdmin","type":"address"}],"name":"changeAdmin","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"depositCount","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"deposits","outputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"tokenAddress","type":"address"},{"internalType":"address","name":"sidechainAddress","type":"address"},{"internalType":"uint32","name":"standard","type":"uint32"},{"internalType":"uint256","name":"tokenNumber","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"implementation","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"pause","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"paused","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"proxyType","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":true,"inputs":[],"name":"registry","outputs":[{"internalType":"contract Registry","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"removeAdmin","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"unpause","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"_newProxyTo","type":"address"}],"name":"updateProxyTo","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"_registry","type":"address"}],"name":"updateRegistry","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"withdrawals","outputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"tokenAddress","type":"address"},{"internalType":"uint256","name":"tokenNumber","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"}]')
contract = web3.eth.contract(address=address, abi = abi)

In [21]:
df = pd.DataFrame(block_dict)

In [22]:
block_data = pd.DataFrame()

In [23]:
def handle_event(event):
    print(Web3.toJSON(event))
    block_dict = {
    'number': [block.number],
    'timestamp': [block.timestamp],
    'difficulty': [block.difficulty],
    'baseFeePerGas': [block.baseFeePerGas],
    'gasLimit':[block.gasLimit],
    'gasUsed': [block.gasUsed],
    'hash': [web3.toHex(block.hash)],
    'miner': [block.miner],
    'size': [block.size],
    'nonce': [web3.toHex(block.nonce)]
}
    df = pd.DataFrame(block_dict)
    block_data = block_data.append(df)

In [24]:
async def log_loop(event_filter, poll_interval):
    while True:
        for PairCreated in event_filter.get_new_entries():
            handle_event(PairCreated)
        await asyncio.sleep(poll_interval)

In [59]:
event_filter.get_new_entries()

[AttributeDict({'args': AttributeDict({'_depositId': 1789277,
   '_owner': '0x0e7868c8fC08129F01d8A65f6fbee9C851fad5E6',
   '_tokenAddress': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   '_sidechainAddress': '0xc99a6A985eD2Cac1ef41640596C5A5f9F4E19Ef5',
   '_standard': 20,
   '_tokenNumber': 335000000000000000}),
  'event': 'TokenDeposited',
  'logIndex': 64,
  'transactionIndex': 48,
  'transactionHash': HexBytes('0x64f258e75a7b7f6a724f145da18297e26c5100004235135b9ba1e96efe9316c3'),
  'address': '0x1A2a1c938CE3eC39b6D47113c7955bAa9DD454F2',
  'blockHash': HexBytes('0x864868876f805b55e0109fe5011f05dabc5daaed28c1d9b3484516ec71243620'),
  'blockNumber': 13444808})]

In [56]:
for x in event_filter.get_new_entries():
    print(x.event)

TokenDeposited
TokenDeposited


In [42]:
contract.web3.eth.get_block('latest', full_transactions=True)

AttributeDict({'baseFeePerGas': 66499668194,
 'difficulty': 9428684940589309,
 'extraData': HexBytes('0x65746865726d696e652d6575726f70652d6e6f72746831'),
 'gasLimit': 29999972,
 'gasUsed': 10951023,
 'hash': HexBytes('0xb8042d237e6c75afc387f1051dd505d1c4ae78298d8504f29feb12aea5ba85d2'),
 'logsBloom': HexBytes('0x0222885a607092838ca03570c0009144050019000c002090c01568400460c151004ca0080450c004444a6ae11802094a0be4d0103c25212007002d000f20694975901253100c438b08618228b13086a0404a2314114502a00088a042c0cb0000268a6000020220c10442fa9020081cb066229488151a0c4408402295600da0105812b22083802a51badb00cd07002144c9c09629010d20080000a04a001600101a7910487d8020002191a9b2820ac487022470442382120c0000000a1ab0d071231d7442c70810c860059769040b094b0958a70416085070069d41820ecd60081131202a023002c01a011429345520e212881105a1a1954140125b21d8200005'),
 'miner': '0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8',
 'mixHash': HexBytes('0xe064dec848bdb805b10260b08ece4ca4f34d4cdb83f2ef3c40270411cfd56d90'),
 'nonce': HexBytes('0x

In [28]:
event_filter = contract.events.TokenDeposited.createFilter(fromBlock='latest')
#block_filter = web3.eth.filter('latest')
#tx_filter = web3.eth.filter('pending')
loop = asyncio.get_event_loop()

try:
    loop.run_until_complete(
    asyncio.gather(log_loop(event_filter,2)))
    print(block_filter)
finally:
    loop.close()

RuntimeError: Cannot close a running event loop

{"args": {"_depositId": 1789254, "_owner": "0x3b419D2beA6Ec5093A5Ff050BFC499155dAaaE9a", "_tokenAddress": "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2", "_sidechainAddress": "0xc99a6A985eD2Cac1ef41640596C5A5f9F4E19Ef5", "_standard": 20, "_tokenNumber": 570000000000000000}, "event": "TokenDeposited", "logIndex": 301, "transactionIndex": 156, "transactionHash": "0x9462f516d00627337167a3bcc89d53532e70652de2165354f1da597df0da99dc", "address": "0x1A2a1c938CE3eC39b6D47113c7955bAa9DD454F2", "blockHash": "0x712376f49666a85aa11c0c4ea16808dac06be01667a9e04ace8b15724e6ee52b", "blockNumber": 13444777}
{"args": {"_depositId": 1789254, "_owner": "0x3b419D2beA6Ec5093A5Ff050BFC499155dAaaE9a", "_tokenAddress": "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2", "_sidechainAddress": "0xc99a6A985eD2Cac1ef41640596C5A5f9F4E19Ef5", "_standard": 20, "_tokenNumber": 570000000000000000}, "event": "TokenDeposited", "logIndex": 301, "transactionIndex": 156, "transactionHash": "0x9462f516d00627337167a3bcc89d53532e70652d

In [37]:
block_data

""


In [38]:
df

""


In [9]:
web3.provider.isConnected()

True

In [ ]:
web3.provider.

In [8]:
web3.currentProvider.connection.close()

AttributeError: 'Web3' object has no attribute 'currentProvider'

{"args": {"_depositId": 1789142, "_owner": "0xbb71408FBf558b4A16c2098B3b938349BE942868", "_tokenAddress": "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2", "_sidechainAddress": "0xc99a6A985eD2Cac1ef41640596C5A5f9F4E19Ef5", "_standard": 20, "_tokenNumber": 130000000000000000}, "event": "TokenDeposited", "logIndex": 177, "transactionIndex": 106, "transactionHash": "0xc19c885869931bee9d831ea38b89fe933ff7fb34524ca5bba67e8ad34ba4520d", "address": "0x1A2a1c938CE3eC39b6D47113c7955bAa9DD454F2", "blockHash": "0x60a7884ee358eac7766fbce3d7b9558823d286a4c14b62df2c76e6361a7f160b", "blockNumber": 13444672}
{"args": {"_depositId": 1789143, "_owner": "0xa9D8e51FB926F4fE3991fe34952C6628B84d59d6", "_tokenAddress": "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2", "_sidechainAddress": "0xc99a6A985eD2Cac1ef41640596C5A5f9F4E19Ef5", "_standard": 20, "_tokenNumber": 360000000000000000}, "event": "TokenDeposited", "logIndex": 218, "transactionIndex": 121, "transactionHash": "0xd18071c78107d70c852e68edd2d3070a34999803

In [ ]:
topic = 'newBlockHeaders'

In [ ]:
web3ws.eth

In [ ]:
web3ws.eth.get_block('latest')

In [ ]:
suscription = web3ws.eth.suscribe(topic)

In [ ]:
suscription.on('data', txHash)

In [ ]:
#latestBlock
block = web3.eth.getBlock('latest', full_transactions=True)
number = block.number
print(f'Searching block {number}')

In [ ]:
block_dict = {
    'number': [block.number],
    'timestamp': [block.timestamp],
    'difficulty': [block.difficulty],
    'baseFeePerGas': [block.baseFeePerGas],
    'gasLimit':[block.gasLimit],
    'gasUsed': [block.gasUsed],
    'hash': [web3.toHex(block.hash)],
    'miner': [block.miner],
    'size': [block.size],
    'nonce': [web3.toHex(block.nonce)]
}

In [ ]:
# DataFrame conversion
block_data = pd.DataFrame(block_dict)

trx_data = pd.DataFrame.from_dict(block.transactions)

In [ ]:
# convert hash into readable hex format
trx_data.r = trx_data.r.apply(lambda x: web3.toHex(x))
trx_data.s = trx_data.s.apply(lambda x: web3.toHex(x))
trx_data.hash = trx_data.hash.apply(lambda x: web3.toHex(x))
trx_data.blockHash = trx_data.blockHash.apply(lambda x: web3.toHex(x))

In [ ]:
# Basic cleansing
def transform(df):
    df.columns = df.columns.str.strip().str.replace(' ', '_').str.upper().str.replace('(','',regex=True)
    objs = df.select_dtypes(['object'])
    for i in objs.columns:
        try:
            df[i] = i.apply(lambda x: x.str.strip())
        except:
            pass
    
    return df

In [ ]:
transform(block_data)
transform(trx_data)

In [ ]:
# Data Load Process

In [ ]:
# Variable definitions
S3_BUCKET = 'blockchainsp'
ETH = 'eth'
BLOCKMETADATA = 'blockmetadata'
BLOCKTRANSACTIONS = 'blocktransactions'
YEAR = str(datetime.fromtimestamp(block.timestamp).year)
MONTH = str(datetime.fromtimestamp(block.timestamp).month)
DAY = str(datetime.fromtimestamp(block.timestamp).day)

In [ ]:
os.path.exists(os.path.abspath(os.path.join('data',ETH,BLOCKTRANSACTIONS,YEAR,MONTH,DAY)))

In [ ]:
os.listdir(os.path.join('data',ETH,BLOCKTRANSACTIONS,YEAR,MONTH,DAY))

In [ ]:
def create_path(path_to_create, delete_first=True):
    ''' Creates necessary folder paths in where to export files
    '''
    import shutil
    if delete_first:
        if os.path.exists(path_to_create):
            shutil.rmtree(path_to_create)
    if not os.path.exists(path_to_create):
        os.makedirs(path_to_create)

    if os.path.exists(os.path.abspath(path_to_create)):
        print(f'{path_to_create} succesfully created')

In [ ]:
create_path(os.path.join('data',ETH,BLOCKTRANSACTIONS,YEAR,MONTH,DAY), delete_first = False)
create_path(os.path.join('data',ETH,BLOCKMETADATA,YEAR,MONTH,DAY), delete_first = False)

In [ ]:
# Define filenames
block_data_filename = f'eth_blockmetadata_{number}_{block.timestamp}'
trx_data_filename = f'eth_blocktransactions_{number}_{block.timestamp}'

In [ ]:
# Local Data dumps
trx_data.to_csv(os.path.join('data',ETH,BLOCKTRANSACTIONS, YEAR, MONTH, DAY, f'{trx_data_filename}.csv'), index=False, sep = ',', encoding='utf-8')
#trx_data.to_json(os.path.join('data',f'{trx_data_filename}.json'), orient='records', default_handler=str)
block_data.to_csv(os.path.join('data',ETH,BLOCKMETADATA, YEAR, MONTH, DAY,f'{block_data_filename}.csv'), index=False, sep = ',', encoding='utf-8')
#block_data.to_json(os.path.join('data',f'{block_data_filename}.json'), orient='records', default_handler=str)


# S3 Data dumps
S3_BUCKET = 'blockchainsp'

s3_services.upload(os.path.join('data',ETH,BLOCKTRANSACTIONS, YEAR, MONTH, DAY,f'{trx_data_filename}.csv'), S3_BUCKET, os.path.join(ETH,BLOCKTRANSACTIONS, YEAR, MONTH, DAY,f'{trx_data_filename}.csv'))
#s3_services.upload(os.path.join('data',f'{trx_data_filename}.csv'), S3_BUCKET, f'{trx_data_filename}.json')
s3_services.upload(os.path.join('data',ETH,BLOCKMETADATA, YEAR, MONTH, DAY,f'{block_data_filename}.csv'), S3_BUCKET, os.path.join(ETH,BLOCKMETADATA, YEAR, MONTH, DAY,f'{block_data_filename}.csv'))
#s3_services.upload(os.path.join('data',f'{block_data_filename}.csv'), S3_BUCKET, f'{block_data_filename}.json')